## Import Libraries

In [1]:
# !pip install pyconll

In [2]:
from conllu import parse_incr
import os
import sys
import re
import pyconll
# import tensorflow as tf 

from transformers import AutoTokenizer, DataCollatorForTokenClassification
from datasets import Dataset
from transformers.keras_callbacks import KerasMetricCallback
from transformers import create_optimizer
from transformers import TFAutoModelForTokenClassification


import warnings
warnings.filterwarnings("ignore", message=".* seems not to be NE tag.")



2025-05-09 04:37:23.821467: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-09 04:37:24.179702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746754644.355922   51234 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746754644.395864   51234 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746754644.719224   51234 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
BATCH_SIZE = 2
EPOCHS = 2
SEED = 42

## login huggingface hub

In [4]:
# from huggingface_hub import notebook_login

# notebook_login()

## Load Data

In [5]:
with open("Arabic_POS.conllu", "r", encoding="utf-8") as f:
    count = 0
    for tokenlist in parse_incr(f):
        print(tokenlist)
        count += 1
        if count == 10:
            break

TokenList<برلين, ترفض, حصول, شركة, اميركية, على, رخصة, تصنيع, دبابة, ", ليوبارد, ", الالمانية, metadata={newdoc id: "afp.20000715.0075", newpar id: "afp.20000715.0075:p1", sent_id: "afp.20000715.0075:p1u1", text: "برلين ترفض حصول شركة اميركية على رخصة تصنيع دبابة "ليوبارد" الالمانية"}>
TokenList<برلين, 15, -, 7, (, اف, ب, ), -, افادت, صحيفة, الاحد, الالمانية, ", ويلت, ام, سونتاغ, ", في, عددها, عدد, ها, الصادر, غدا, ،, ان, المستشار, غيرهارد, شرودر, يرفض, حصول, المجموعة, الاميركية, ", جنرال, ديناميكس, ", على, رخصة, لتصنيع, ل, تصنيع, الدبابة, الالمانية, ", ليوبارد, 2, ", عبر, شراء, المجموعة, الحكومية, الاسبانية, للاسلحة, ل, الأسلحة, ", سانتا, بربارة, ", ., metadata={newpar id: "afp.20000715.0075:p2", sent_id: "afp.20000715.0075:p2u1", text: "برلين 15-7 (اف ب) - افادت صحيفة الاحد الالمانية "ويلت ام سونتاغ" في عددها الصادر غدا، ان المستشار غيرهارد شرودر يرفض حصول المجموعة الاميركية "جنرال ديناميكس" على رخصة لتصنيع الدبابة الالمانية "ليوبارد 2" عبر شراء المجموعة الحكومية الاسبانية للاسلحة "س

In [6]:
data = pyconll.load_from_file('Arabic_POS.conllu')

In [7]:
sentences = []
labels = []
for sentence in data:
  sentences.append([token.form for token in sentence])
  labels.append([token.upos for token in sentence])

In [8]:
unique_labels = set(label for sublist in labels for label in sublist if label is not None)
print(unique_labels)

{'DET', 'SCONJ', 'X', 'CCONJ', 'PART', 'ADV', 'PROPN', 'VERB', 'AUX', 'PRON', 'PUNCT', 'NUM', 'INTJ', 'ADP', 'NOUN', 'ADJ', 'SYM'}


In [9]:
labels_list = ['CCONJ', 'VERB', 'ADP', 'DET', 'NUM', 'PRON', 'AUX', 'PROPN', 'PUNCT', 'PART', 'NOUN', 'SYM', 'SCONJ', 'INTJ', 'ADV', 'ADJ', 'X', 'None']

In [10]:
labels[1]

['X',
 'NUM',
 'PUNCT',
 'NUM',
 'PUNCT',
 'X',
 'X',
 'PUNCT',
 'PUNCT',
 'VERB',
 'NOUN',
 'NOUN',
 'ADJ',
 'PUNCT',
 'X',
 'X',
 'X',
 'PUNCT',
 'ADP',
 None,
 'NOUN',
 'PRON',
 'ADJ',
 'NOUN',
 'PUNCT',
 'SCONJ',
 'NOUN',
 'X',
 'X',
 'VERB',
 'NOUN',
 'NOUN',
 'ADJ',
 'PUNCT',
 'X',
 'X',
 'PUNCT',
 'ADP',
 'NOUN',
 None,
 'ADP',
 'NOUN',
 'NOUN',
 'ADJ',
 'PUNCT',
 'X',
 'NUM',
 'PUNCT',
 'ADP',
 'NOUN',
 'NOUN',
 'ADJ',
 'ADJ',
 None,
 'ADP',
 'NOUN',
 'PUNCT',
 'X',
 'X',
 'PUNCT',
 'PUNCT']

In [11]:
sentences[1]

['برلين',
 '15',
 '-',
 '7',
 '(',
 'اف',
 'ب',
 ')',
 '-',
 'افادت',
 'صحيفة',
 'الاحد',
 'الالمانية',
 '"',
 'ويلت',
 'ام',
 'سونتاغ',
 '"',
 'في',
 'عددها',
 'عدد',
 'ها',
 'الصادر',
 'غدا',
 '،',
 'ان',
 'المستشار',
 'غيرهارد',
 'شرودر',
 'يرفض',
 'حصول',
 'المجموعة',
 'الاميركية',
 '"',
 'جنرال',
 'ديناميكس',
 '"',
 'على',
 'رخصة',
 'لتصنيع',
 'ل',
 'تصنيع',
 'الدبابة',
 'الالمانية',
 '"',
 'ليوبارد',
 '2',
 '"',
 'عبر',
 'شراء',
 'المجموعة',
 'الحكومية',
 'الاسبانية',
 'للاسلحة',
 'ل',
 'الأسلحة',
 '"',
 'سانتا',
 'بربارة',
 '"',
 '.']

## preprocessing the data 

In [13]:
sentense_id = []
for i in range (len(sentences)):
  sentense_id.append(i)

In [14]:
sentense_id[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
label_map = {label: i for i, label in enumerate(labels_list)}
id2label = {i: l for i, l in enumerate(labels_list)}

# Ensure all 'None' values are replaced with a valid key
labels = [
    [tag if tag is not None else "None" for tag in tag_sequence]
    for tag_sequence in labels
]

# Build label_ids using the updated labels
label_ids = [
    [label_map[tag] for tag in tag_sequence]
    for tag_sequence in labels
]
label_ids[0]

[16, 1, 10, 10, 15, 2, 10, 10, 10, 8, 16, 8, 15]

In [16]:
labels[0]

['X',
 'VERB',
 'NOUN',
 'NOUN',
 'ADJ',
 'ADP',
 'NOUN',
 'NOUN',
 'NOUN',
 'PUNCT',
 'X',
 'PUNCT',
 'ADJ']

In [17]:
data_dict = {
    'id': sentense_id,
    'ner_tags': label_ids,
    'tokens': sentences,
}

In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [19]:
# 1. Initialize tokenizer with correct Arabic model
# tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
tokenizer = AutoTokenizer.from_pretrained(
    "aubmindlab/bert-base-arabertv02",
    max_length=128,  # Add max length
    truncation=True
)
# 2. Now create the data collator
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    return_tensors="tf"  
)

In [20]:
data_dict = Dataset.from_dict(data_dict)

In [21]:
tokenized_data = data_dict.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/6075 [00:00<?, ? examples/s]

In [22]:
train_test_split = tokenized_data.train_test_split(test_size=0.2, seed=SEED)

In [23]:
tokenized_data_train = train_test_split['train']
tokenized_data_test = train_test_split['test']

In [24]:
import evaluate
import numpy as np
num_train_steps = (len(train_test_split["train"]) // BATCH_SIZE) * EPOCHS

seqeval = evaluate.load("seqeval")

In [25]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [26]:
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [27]:
model = TFAutoModelForTokenClassification.from_pretrained(
    "aubmindlab/bert-base-arabertv02", 
    num_labels=len(labels_list),
    id2label=id2label, 
    label2id=label_map
)

I0000 00:00:1746754687.650913   51234 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4269 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are new

In [28]:
tf_train_set = model.prepare_tf_dataset(
    train_test_split["train"],
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    train_test_split["test"],
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

In [29]:
model.compile(optimizer=optimizer)

In [30]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="Arabic_POS_classifer",
    hub_model_id="Israaabdelghany/Arabic_POS_classifer",
    tokenizer=tokenizer,
)

/home/israa/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/home/israa/Desktop/Arabic_POS_Tagging/Arabic_POS_model is already a clone of https://huggingface.co/Israaabdelghany/Arabic_POS. Make sure you pull the latest changes with `repo.git_pull()`.


In [31]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback, push_to_hub_callback]

In [35]:
# Reduce batch size to avoid OOM errors
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=EPOCHS, callbacks=callbacks, batch_size=3)

Epoch 1/2
2430/2430 [==============================] - 587s 241ms/step - loss: 0.0670 - val_loss: 0.1015 - precision: 0.9651 - recall: 0.9605 - f1: 0.9628 - accuracy: 0.9725
Epoch 2/2
2430/2430 [==============================] - ETA: 0s - loss: 0.0665

2025-05-09 05:43:16.624128: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2430/2430 [==============================] - 601s 247ms/step - loss: 0.0665 - val_loss: 0.1015 - precision: 0.9651 - recall: 0.9605 - f1: 0.9628 - accuracy: 0.9725


In [ ]:
model.save_pretrained("Arabic_POS_classifer", id2label=id2label, label2id=label_map)

In [ ]:
model.push_to_hub("Arabic_POS_classifer")
tokenizer.push_to_hub("Arabic_POS_classifer")

In [ ]:
text = "برلين ترفض حصول شركة امريكية على رخصة"

In [ ]:
from transformers import pipeline

classifier = pipeline("ner", model="MariamOsama3/Mariam_classifer2")
classifier(text)